## Import libraries

In [1]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from keras.models import Model, load_model
from keras.layers import Flatten, Dense, Lambda
from keras.applications.resnet50 import ResNet50
from keras.optimizers import Adam
from keras.preprocessing import image

Using TensorFlow backend.


## Set variables and import data

In [2]:
gen = image.ImageDataGenerator()
batch_size = 64

In [3]:
batches = gen.flow_from_directory('task3_data_final/train',
                                  target_size=(224,224),
                                  class_mode='categorical',
                                  shuffle=True,
                                  batch_size=batch_size)

Found 34304 images belonging to 7 classes.


In [11]:
val_batches = gen.flow_from_directory('task3_data_final/valid',
                                      target_size=(224,224),
                                      class_mode='categorical',
                                      shuffle=False,
                                      batch_size=batch_size)

Found 8584 images belonging to 7 classes.


## Import, configure, and compile model

In [9]:
resnet = ResNet50(include_top=True, weights='imagenet', input_tensor=None, input_shape=(224,224,3), pooling=None)

x = resnet.layers[-2].output
output_layer = Dense(7, activation='softmax', name='predictions')(x)

resnet2 = Model(inputs=resnet.input, outputs=output_layer)

resnet2.compile(optimizer=Adam(lr=0.000015),
                loss='categorical_crossentropy', metrics=['accuracy'])

## Train and save model

In [10]:
resnet2.fit_generator(batches,
                   steps_per_epoch = batches.samples // batch_size,
                   validation_data = val_batches, 
                   validation_steps = val_batches.samples // batch_size,
                   epochs = 1)

Epoch 1/1
536/536 [==============================] - 281s 525ms/step - loss: 0.7592 - acc: 0.7234 - val_loss: 0.7210 - val_acc: 0.7381


In [17]:
resnet2.save('task3_models/resnet_last.h5')

## Confusion and classification matrices

In [12]:
Y_pred = resnet2.predict_generator(val_batches, val_batches.samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)

In [13]:
print('Confusion Matrix')
conf_matrix = confusion_matrix(val_batches.classes, y_pred)
print(conf_matrix)
np.savetxt("conf_matrix.csv", conf_matrix, delimiter=",", fmt="%s")

Confusion Matrix
[[ 504   73   92   53   42    4    0]
 [  61  618   40   17   30   23   35]
 [ 102   80 1200   35  221  122    0]
 [  64   46   13  387    7   26    1]
 [  82   49  249   14 1184  193   13]
 [   9   29  128   29  233 2237   15]
 [   0    1    3    0    8    6  206]]


In [14]:
print('Classification Report')
target_names = ['AKIEC', 'BCC', 'BKL', 'DF', 'MEL', 'NV', 'VASC']
print(classification_report(val_batches.classes, y_pred, target_names=target_names))

Classification Report
             precision    recall  f1-score   support

      AKIEC       0.61      0.66      0.63       768
        BCC       0.69      0.75      0.72       824
        BKL       0.70      0.68      0.69      1760
         DF       0.72      0.71      0.72       544
        MEL       0.69      0.66      0.67      1784
         NV       0.86      0.83      0.85      2680
       VASC       0.76      0.92      0.83       224

avg / total       0.74      0.74      0.74      8584



## Load model, generate predictions, and output results

In [ ]:
resnet2 = load_model('task3_models/resnet_last.h5')

In [15]:
# Validation predictions
test = gen.flow_from_directory('task3_data_final/test_valid',
                                target_size=(224,224),
                                class_mode='categorical',
                                shuffle=False,
                                batch_size=batch_size)

Found 193 images belonging to 1 classes.


In [16]:
f_names = np.array(test.filenames)
predictions = np.array(resnet2.predict_generator(test))
resnet_predictions = np.column_stack((f_names, predictions))
resnet_predictions[:,0] = np.char.replace(resnet_predictions[:,0], "test/", "")
resnet_predictions[:,0] = np.char.replace(resnet_predictions[:,0], ".jpg", "")
resnet_predictions = np.insert(resnet_predictions, 0, np.array(("image","AKIEC","BCC","BKL","DF","MEL","NV","VASC")), 0)
np.savetxt("resnet_valid_predictions.csv", resnet_predictions, delimiter=",", fmt="%s")

In [18]:
# Final predictions
test = gen.flow_from_directory('task3_data_final/test_final',
                                target_size=(224,224),
                                class_mode='categorical',
                                shuffle=False,
                                batch_size=batch_size)

Found 1512 images belonging to 1 classes.


In [19]:
f_names = np.array(test.filenames)
predictions = np.array(resnet2.predict_generator(test))
resnet_predictions = np.column_stack((f_names, predictions))
resnet_predictions[:,0] = np.char.replace(resnet_predictions[:,0], "test/", "")
resnet_predictions[:,0] = np.char.replace(resnet_predictions[:,0], ".jpg", "")
resnet_predictions = np.insert(resnet_predictions, 0, np.array(("image","AKIEC","BCC","BKL","DF","MEL","NV","VASC")), 0)
np.savetxt("resnet_final_predictions.csv", resnet_predictions, delimiter=",", fmt="%s")